<div class="licence">
<span>Licence CC BY-NC-ND</span>
<span>Valérie Roy</span>
<span><img src="../media/ensmp-25-alpha.png" /></span>
</div>

In [ ]:
import numpy as np
import pandas as pd


# pivoting tables for grouping

we'll study 2 examples:

* **category-based**:
  * 'category' means a data is among a (small) *finite set of possible values*
  * example: 'male' or 'female'; 'lower', 'middle', 'upper'

* **bin-based**: 
  * for valued data, we can always turn them into a category
  * by defining 'bins' - i.e. *intervals*
  * to turn that data into a category

## example 1: category-based (Titanic survival rate by class and sex)
we have a **dataset** about the sinking of **titanic** that gives (among others) for each **passenger**:  
- his/her **survival status** (yes or no), **sex**, **class** (first, second, third), and **age**

In [ ]:
# importing the whole dataset
df = pd.read_csv('titanic.csv')
df.head(3)

that's a little too much ! we do not need all those columns...

### keeping the **interesting columns** 
   - using *read_csv*'s `usecols` parameter 

In [ ]:
df = pd.read_csv(
    'titanic.csv', 
    usecols=['Survived', 'Pclass', 'Sex', 'Age'])

In [ ]:
df.head(3)

### some information about the data

In [ ]:
#  number of passengers in each class
print(df['Pclass'].value_counts())

print()
#  number of passengers of each sex
print(df['Sex'].value_counts())

In [ ]:
from matplotlib import pyplot as plt
plt.figure()
plt.hist(df['Age'], bins=15);

### suppose you want to know
   - the **survival rate** depending on the **sex** and the **class** 

this can be done with the *pandas.DataFrame.pivot_table* **method**

we are going to

* group data that have the same **sex** index and the same **Pclass** column
* and on each group, compute the **mean** of their **survival status**

in terms of the `pivot_table` parameter, this translates into
   - the **value** to be **aggregated** here **Survived**
   - the **aggregation** function here the *numpy.mean*
   - the **key** to be the **index**   here the column **sex**
   - the **key** to be the **column**  here the column **Pclass**

### the **survival rate** depending on the **sex** and the **class** 

the result of pivot_table is a new dataframe

In [ ]:
df.pivot_table('Survived', index='Sex', columns='Pclass', aggfunc=np.mean)
# the new table with sex as index and the three classes as columns
# where the values are the means

on the Titanic, it was better to be a first class woman than a third class man...

## example 2: bin-based (survival rate by age group)   - you want to **compute** the **survival rate** by **age group**
   - but we do not have **age group**
   - so we must **pack** the ages in **bins** representing **age group**

   - you want to **compute** the **survival rate** by **age group**
   - but we do not have **age group**
   - so we must **pack** the ages in **bins** representing **age groupe**
   

   - we create **bins** of ages and **names** for those **bins**

In [ ]:
# her we define the threshhold or our age groups
age_groups = [0, 11, 17, 25, 35, 45, 55, 65, 100]  # from 0 to 11, from 11 to 17...

# and for convenience we give each of them a handy label
age_group_names = ['<11', '11-17', '17-25', '25-35', '35-45', '45-55', '55-65', '>65']


- we create a new category **column**, that gives the person's **age group** 
- using the *pandas.cut* **method** with the **bins** and the **names** parameters
- we just **add** the column in our data frame

In [ ]:
df['Age group'] = pd.cut(
    df['Age'], bins=age_groups, labels=age_group_names)
# we have "bin" the Age column into discrete intervals
df.head()

- so now we have a category column
- so we use *pivot_table*
- to compute a new **data frame** with the  **survival rate** by **age group**

In [ ]:
# this time on a  single column 'sex'
df.pivot_table('Survived', index=['Sex'], columns='Age group', aggfunc=np.mean) 

women and little boys first !

   - if we're just interested in the global survival rate by age-group but not by sex
   - then just don't specify an index parameter (or say index=[])

In [ ]:
df.pivot_table('Survived', index=[], columns='Age group', aggfunc=np.mean)

   - we **do not need** to **add** a column to the **data frame**
   - here we  **pass** the number of **bins** and their **names**
   - pandas will automatically split the 'age' column in 3 equal intervals

In [ ]:
col = pd.cut(df['Age'], 3, labels=['child', 'adult', 'old'])
print(col.head(3))
print()
print(col.tail(3))

same use of pivot_table as before, except that *col* is not a part of df

In [ ]:
df.pivot_table('Survived', index=['Sex', 'Pclass'], columns=col, aggfunc=np.mean)

## digression: tweaking data

In [ ]:
df = pd.read_csv('titanic.csv', usecols=('Pclass', 'Age', 'Sex'))

example of changing the **data**
  - for example, we want to **replace** the **number** code of classes
  - by a more user-friendly **name** ('first' instead of 1)
  

In [ ]:
# we start with creating a mask that is
# True on all people in first class

mask = (df['Pclass'] == 1)

In [ ]:
# we have a mask of indexes
mask.head(3)

- we **locate** the **true** values
- and replace their **Pclass** column **value** by the string **'first'**
- always use **loc** or **iloc**  
  **never** use a classical **array** assignement

In [ ]:
df.loc[ mask, 'Pclass'] = 'first'

In [ ]:
# and so on
df.loc[ df['Pclass'] == 2, 'Pclass'] = 'second'
df.loc[ df['Pclass'] == 3, 'Pclass'] = 'third'

In [ ]:
df.head(3)